In [1]:
%load_ext autoreload

%autoreload 2

This notebook demonstrates how to use the local optimization procedure based on the topological descriptors by Kozlov et al. and EMT, as it is implemented in NPL.

In [2]:
import Core.Nanoparticle as NP
import Core.GOSearch as GOS
from LocalOpt.LocalOptimization import local_optimization

import Core.EnergyCalculator as EC

from ase.visualize import view
import pickle


In [4]:
"""Create a list of particles with fixed composition and shape (truncated octahedron), but random ordering
and calculate the energy using EMT
"""

def create_octahedron_training_set(n_particles, height, trunc, stoichiometry):
    emt_calculator = EC.EMTCalculator(fmax=0.03, steps=1000)
    
    training_set = []
    for i in range(n_particles):
        p = NP.Nanoparticle()
        p.truncated_octahedron(height, trunc, stoichiometry)
        emt_calculator.compute_energy(p)
        training_set.append(p)
        
    return training_set

In [5]:
"""Create one randomly ordered start particle"""

def create_start_particle(height, trunc, stoichiometry):
    start_particle = NP.Nanoparticle()
    start_particle.truncated_octahedron(height, trunc, stoichiometry)
    return start_particle

In [6]:
"""Create the training set with 30 particles"""
stoichiometry={'Pt' : 55, 'Au' : 24}

training_set = create_octahedron_training_set(30, 5, 1, stoichiometry)

      Step     Time          Energy         fmax
BFGS:    0 09:39:49       30.664922        1.7428
BFGS:    1 09:39:49       29.786630        1.3785
BFGS:    2 09:39:49       28.645306        1.5314
BFGS:    3 09:39:49       28.497148        0.6162
BFGS:    4 09:39:49       28.426334        0.4169
BFGS:    5 09:39:49       28.173652        0.4000
BFGS:    6 09:39:49       28.144334        0.3644
BFGS:    7 09:39:49       28.087287        0.1537
BFGS:    8 09:39:49       28.073885        0.1314
BFGS:    9 09:39:49       28.063708        0.1282
BFGS:   10 09:39:49       28.056325        0.1083
BFGS:   11 09:39:49       28.050178        0.0763
BFGS:   12 09:39:49       28.046872        0.0720
BFGS:   13 09:39:49       28.044780        0.0523
BFGS:   14 09:39:49       28.043128        0.0521
BFGS:   15 09:39:49       28.041912        0.0475
BFGS:   16 09:39:49       28.041290        0.0300
BFGS:   17 09:39:49       28.040962        0.0265
      Step     Time          Energy         fmax
BF

In [9]:
"""First we create an Object for a global optimization search and pass to it references to functions that we
want to use for optimizing (local_optimization) and for creating a start configuration (create_start_particle)."""

guided_MC_search = GOS.GuidedSearch(local_optimization, create_start_particle)

"""We then have it fit the topological descriptors to the training set we just created"""
symbols = list(stoichiometry.keys()) #['Pt', 'Au']
guided_MC_search.fit_energy_expression(training_set, symbols)

"""We start the optimization by calling the run_multiple simulations function. In this case we want to do two
runs (n_sim_runs = 2). We can pass parameters to both functions (local optmization & create start particle) that will
be used when the optimization is actually started. In this case we make sure, that the start particle has the same
shape as the particles in the training set."""

n_sim_runs = 2
args_start = [5, 1, stoichiometry] # height, trunc, composition -> parameters of the create_start_particle function
results, run_times = guided_MC_search.run_multiple_simulations(n_sim_runs, args_start=args_start)

[-2.61515798e-01  2.75929327e-02  9.95775587e-01  4.29902608e-01
 -2.18488166e-12  1.45262964e-12 -9.50007374e-15  1.93031238e-15
 -1.85766397e-15 -5.38971180e-29  9.86093151e-01  1.01355181e+00
 -1.18009157e-32  1.08573991e+00 -9.02674760e-47  0.00000000e+00
  1.21364120e+00]
Coef symbol_a: Au
Run: 0
Runtime: 0.015198623999822303
Run: 1
Runtime: 0.013091294999867387


In [20]:
"""The results object will be a list of optimization runs. Each optimization run holds the final structure as well
as the energies and the step number of the considered configurations. Run times for every run will be returned as a
list"""
print('Structure : {}'.format(results[0][0]))
print(' ')
print('(Energy, step) pairs:')
print(results[0][1])
print(' ')
print('Runtime in s: {}'.format(run_times[0]))

Structure : <Core.Nanoparticle.Nanoparticle object at 0x7fa57b0c4748>
 
(Energy, step) pairs:
[(28.74106472319444, 0), (28.439983811787414, 1), (28.110732342908783, 2), (27.764853897166496, 3), (27.49194354323108, 4), (27.1742356549604, 5), (26.87315474355338, 6), (26.6637734578778, 7), (26.52736026400909, 8), (26.37432009327672, 9), (26.249450480015955, 10), (26.197548958562063, 11), (26.145647437108167, 12), (26.134103856500218, 13), (26.094389718420665, 14), (26.094389718420665, 15)]
 
Runtime in s: 0.015198623999822303


In [21]:
"""Display the final structure of the first run"""
p = results[0][0]
view(p.get_ase_atoms())

In [22]:
"""This is not the optimal solution. We can use the Basin Hopping instead of the local optimization to find it.
For this we have to import the run_basin_hopping function and pass some additional start parameters to the search
object"""

from BH.BasinHopping import run_basin_hopping

guided_MC_search = GOS.GuidedSearch(run_basin_hopping, create_start_particle)

"""Fit energy expression"""
symbols = list(stoichiometry.keys()) #['Pt', 'Au']
guided_MC_search.fit_energy_expression(training_set, symbols)

"""For the Basin Hopping we also have to pass the number of (not necessarily distinct) basins we want to expore 
(n_hopping_attempts) and how strong we want to perturbate the locally optimal solutions (n_exchanges). We can
pass them the same way as for the start particle. run_basin_hopping and local optimization also take different
parameters. Fortunately their function signatures are the same for the first three parameters, so we can reuse
the GuidedSearch class for both optmization functions. The remaining parameters can be passed simply as list, 
which should prove useful if the function signature changes at some point or one ones to implement a different
optimization procedure."""
n_hopping_attempts = 20
n_exchanges = 10
args_bh = [n_hopping_attempts, n_exchanges]

n_sim_runs = 2
args_start = [5, 1, stoichiometry] # height, trunc, composition -> parameters of the create_start_particle function

results, run_times = guided_MC_search.run_multiple_simulations(n_sim_runs, args_start=args_start, args_gm = args_bh)

[-2.61515798e-01  2.75929327e-02  9.95775587e-01  4.29902608e-01
 -2.18488166e-12  1.45262964e-12 -9.50007374e-15  1.93031238e-15
 -1.85766397e-15 -5.38971180e-29  9.86093151e-01  1.01355181e+00
 -1.18009157e-32  1.08573991e+00 -9.02674760e-47  0.00000000e+00
  1.21364120e+00]
Coef symbol_a: Au
Run: 0
Energy after local_opt: 25.904, lowest 25.904
Lowest energy: 25.904
Runtime: 0.30893823899987183
Run: 1
Energy after local_opt: 25.950, lowest 25.950
Lowest energy: 25.904
Runtime: 0.3052431329997489


In [23]:
"""Now we should have the best solution for this system, both runs finished with the same energy"""
p = results[0][0]
view(p.get_ase_atoms())